In [24]:
from bs4 import BeautifulSoup
import requests
import time, os

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [18]:
county_census_info = pd.read_pickle("../Data/county_census_info.pkl")

In [22]:
indices = county_census_info.index
print(indices[0])

('Autauga County', 'Alabama')


In [29]:
cdc_county_website = "https://covid.cdc.gov/covid-data-tracker/#county-view"

driver = webdriver.Chrome(chromedriver)
driver.get(cdc_county_website)

time.sleep(5)

select = Select(driver.find_element_by_id('list_select_state'))
select.select_by_visible_text('Texas')

select = Select(driver.find_element_by_id('list_select_county'))
select.select_by_visible_text('Travis')

time.sleep(10)

# driver.find_element_by_xpath("//select[id='list_select_state']/option[text()='Texas']").click()


# driver.close()

In [30]:
driver.close()